# 分类算法的评价


> 预测的准确度确实是一个很好很直观的评价指标，但是有时候正确率高并不能代表一个算法就好
> 比如某个地区某天地震的预测，假设我们有一堆的特征作为地震分类的属性，类别只有两个，0不发生地震、1发生地震。一个不加思考的分类器，对每一个测试用例都将类别划分为0，那么它就可能达到99%的正确率，但真的地震来临时，这个分类器毫无察觉，这个分类带来的损失是巨大的。为什么99%的正确率的分类器却不是我们想要的，因为这里数据分布不均衡，类别1的数据太少，完全错分类别1依然可以达到很高的正确率却忽视了我们关注的东西。

## 混淆矩阵

混淆矩阵也称误差矩阵，是表示精度评价的一种标准格式，用n行n列的矩阵形式来表示。

混淆矩阵的每一列代表了预测类别，每一列的总数表示预测为该类别的数据的数目；每一行代表了数据的真实归属类别，每一行的数据总数表示该类别的数据实例的数目。

|   | 预测类别1 | 预测类别2 | 预测类别3 |
| :-: | :-: | :-: | :-: |
| 真实类别1 | / | / | / |
| 真实类别2 | / | / | / |
| 真实类别3 | / | / | / |

对于二分类问题，则有

|   | 0 | 1 |
| :-: | :-: | :-: |
| 0 | TN(预测negative正确数) | FP(预测positive错误数) |
| 1 | FN(预测negative错误数) | TP(预测positive正确数) |

T表示预测结果和真实值相同，F则相反

N表示`Negative`即0, P表示`Positive`即1



## 精准率和召回率

精准率表示预测结果为1，且预测正确的概率。公式表示为 $Precision=\frac {TP}{TP+FP}$

召回率表示真实的分类中被成功预测的概率。公式表示为 $Recall=\frac {TP}{TP + FN}$

TPR？FPR?

## 实现混淆矩阵，准确率和召回率

In [1]:
import numpy as np
from sklearn import datasets

digits = datasets.load_digits()

X = digits.data
y = digits.target.copy()

# 构造偏斜的数据
y[digits.target==9] = 1
y[digits.target!=9] = 0

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [3]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.97555555555555551

In [4]:
y_log_predict = log_reg.predict(X_test)

In [5]:
# 预测negative正确数
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 0))
TN(y_test, y_log_predict)

403

In [6]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 1))

FP(y_test, y_log_predict)

2

In [7]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 0))

FN(y_test, y_log_predict)

9

In [8]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 1))

TP(y_test, y_log_predict)

36

In [9]:
# 求混淆矩阵
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [10]:
# 精准率
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0
    
precision_score(y_test, y_log_predict)

0.94736842105263153

In [11]:
# 回归率
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0
    
recall_score(y_test, y_log_predict)

0.80000000000000004

## sklearn中的混淆矩阵，精准率和召回率

In [12]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [13]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.94736842105263153

In [14]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.80000000000000004

## F1 Score

精准率和召回率的选取由应用场景决定。有时候更加注重精准率，如股票预测。有时候更加注重召回率，如病人诊断。

但有时候需要同时考虑精准率和召回率，这里引入`F1 Score`, 是精准率和召回率的调和平均值

$
\frac {1}{F1} = \frac {1}{2} (\frac {1}{precision} + \frac {1}{recall})
$

$
F1 = \frac {2 * precision * recall}{precision + recall}
$

In [15]:
def f1_score(precision, recall):
    try:
        return 2 * precision * recall / (precision + recall)
    except:
        return 0.0

In [16]:
precision = 0.5
recall = 0.5
f1_score(precision, recall)

0.5

In [17]:
precision = 0.1
recall = 0.9
f1_score(precision, recall)

0.18000000000000002

In [18]:
precision = 0.0
recall = 1.0
f1_score(precision, recall)

0.0

In [20]:
from sklearn.metrics import f1_score

f1_score(y_test, y_log_predict)

0.86746987951807231